# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

> You do not need to run the following cells if you are running this notebook locally. 

In [11]:
#!pip install -qU langchain langchain-openai langchain-cohere rank_bm25

We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [1]:
#!pip install -qU qdrant-client

We'll also provide our OpenAI key, as well as our Cohere API key.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [3]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [4]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

--2025-03-02 10:25:14--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19628 (19K) [text/plain]
Saving to: ‘john_wick_1.csv’

john_wick_1.csv     100%[===================>]  19.17K  --.-KB/s    in 0s      

2025-03-02 10:25:14 (84.3 MB/s) - ‘john_wick_1.csv’ saved [19628/19628]

--2025-03-02 10:25:15--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14747 (14K) [text/plain]
Sa

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [5]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [6]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2025, 2, 27, 10, 25, 16, 778214)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [7]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [8]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [9]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-3.5-turbo` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [10]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [12]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [ ]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the positive reviews provided.'

In [ ]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review: '/review/rw4854296/?ref_=tt_urv'."

In [ ]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, an ex-hitman comes out of retirement to seek vengeance against the gangsters who killed his dog and took everything from him. His pursuit for revenge leads to intense action, shootouts, and breathtaking fights as he becomes a target for hitmen and bounty hunters. The movie is known for its violent and gripping story with plenty of unstoppable action.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [13]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [14]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [ ]:
bm25_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"People's opinions on the John Wick movies vary. Some people really enjoy the action and style of the movies, while others find them lacking in plot or substance. Overall, whether people like John Wick or not depends on personal preferences."

In [ ]:
bm25_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"I'm sorry, there are no reviews with a rating of 10 in the context provided."

In [ ]:
bm25_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the protagonist, played by Keanu Reeves, is a former assassin seeking revenge for the death of his beloved dog, which was a gift from his late wife. This sets off a chain of events that leads to intense action sequences and a thrilling journey of vengeance.'

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [36]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [37]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [25]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the reviews provided.'

In [26]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review:\n\nReview URL: /review/rw4854296/?ref_=tt_urv'

In [27]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick 2, John Wick is forced out of retirement when a mobster visits him with a marker that needs to be fulfilled. After carrying out the task, a contract is put on him, leading to a series of events where John Wick must fight to survive.'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [15]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [16]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [30]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Yes, people generally liked John Wick based on the reviews provided. The film received praise for its cool action sequences, Keanu Reeves' performance, and overall entertainment value."

In [31]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review: '/review/rw4854296/?ref_=tt_urv'."

In [32]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'John Wick is a retired assassin who seeks revenge after his wife dies and his dog is killed by Russian mobsters. This sets off a series of events where Wick, a super-assassin, takes on the Russian mob after they attack him, leading to extreme violence and action sequences.'

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [17]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [18]:
from langchain_openai import OpenAIEmbeddings

qdrant_client = QdrantClient(location=":memory:")

qdrant_client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=qdrant_client
)

/var/folders/v5/wmnn3myj233d2t2rkgqsrgw80000gn/T/ipykernel_16810/2886526882.py:10: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [19]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [20]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [21]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [38]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, opinions about John Wick seem to be divided. Some people really enjoy the series, while others, like the reviewer with the username "solidabs," did not like John Wick 4 at all. Therefore, it is safe to say that people do not generally agree on whether they like John Wick or not.'

In [39]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review: '/review/rw4854296/?ref_=tt_urv'"

In [40]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, the retired assassin John Wick comes out of retirement to seek vengeance after his dog is killed and his car is stolen, leading to a lot of carnage and him being dragged into an impossible task. He helps Ian McShane take over the Assassin's Guild by flying around to Italy, Canada, and Manhattan and killing numerous assassins."

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [22]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [
  naive_retriever, 
  bm25_retriever, 
#   compression_retriever, 
  parent_document_retriever, 
  multi_query_retriever
]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [23]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [43]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'The majority of people seemed to really enjoy John Wick based on the positive reviews provided.'

In [44]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is one review with a rating of 10. Here is the URL to that review: \'/review/rw4854296/?ref_=tt_urv\' for the movie "John Wick 3".'

In [45]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the main character, portrayed by Keanu Reeves, is an ex-hitman seeking revenge after gangsters kill his dog and steal his car. This leads to a series of intense action sequences as Wick takes on various adversaries in his quest for vengeance.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

> NOTE: You do not need to run this cell if you're running this locally

In [46]:
#!pip install -qU langchain_experimental

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [28]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [29]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [30]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [31]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [32]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [52]:
semantic_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Overall, people generally liked John Wick based on the reviews provided.'

In [53]:
semantic_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for "John Wick 3". Here is the URL to that review: \'/review/rw4854296/?ref_=tt_urv\'.'

In [54]:
semantic_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In "John Wick", the main character, John Wick, seeks revenge on the people who took something he loved from him. Specifically, in the first movie, he comes out of retirement to track down the gangsters that killed his dog and took everything from him. He unleashes a maelstrom of destruction against those who have wronged him.'

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [ ]:
#!pip install -qU ragas langsmith pandas rapidfuzz

In [25]:
from typing_extensions import Annotated, TypedDict
from langchain_openai import ChatOpenAI

# Grade output schema
class RetrievalRelevanceGrade(TypedDict):
  explanation: Annotated[str, ..., "Explain your reasoning for the score"]
  relevant: Annotated[bool, ..., "True if the retrieved documents are relevant to the question, False otherwise"]

# Grade prompt
retrieval_relevance_instructions = """You are a teacher grading a quiz. 

You will be given a QUESTION and a set of FACTS provided by the student. 

Here is the grade criteria to follow:
(1) You goal is to identify FACTS that are completely unrelated to the QUESTION
(2) If the facts contain ANY keywords or semantic meaning related to the question, consider them relevant
(3) It is OK if the facts have SOME information that is unrelated to the question as long as (2) is met

Relevance:
A relevance value of True means that the FACTS contain ANY keywords or semantic meaning related to the QUESTION and are therefore relevant.
A relevance value of False means that the FACTS are completely unrelated to the QUESTION.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Do not state the correct answer at the outset."""

# Grader LLM
retrieval_relevance_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(RetrievalRelevanceGrade, method="json_schema", strict=True)

def retrieval_relevance(inputs: dict, outputs: dict) -> bool:
  """An evaluator for document relevance"""
  doc_string = "\n\n".join(doc.page_content for doc in outputs["context"])
  answer = f"""      FACTS: {doc_string}\nQUESTION: {inputs['question']}"""
  # Run evaluator
  grade = retrieval_relevance_llm.invoke([{"role": "system", "content": retrieval_relevance_instructions}, {"role": "user", "content": answer}])
  return grade["relevant"]

In [ ]:
import pandas as pd
from pathlib import Path

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from ragas.llms import LangchainLLMWrapper
from ragas.testset import TestsetGenerator
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.metrics import LLMContextRecall, ContextEntityRecall

eval_gen_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
eval_judge_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
eval_gen_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

chains = [
  ("naive_retrieval", naive_retrieval_chain),
  ("bm25_retrieval", bm25_retrieval_chain),
  # ("contextual_compression_retrieval", contextual_compression_retrieval_chain),
  ("multi_query_retrieval", multi_query_retrieval_chain),
  ("parent_document_retrieval", parent_document_retrieval_chain),
  ("ensemble_retrieval", ensemble_retrieval_chain),
  # ("semantic_retrieval", semantic_retrieval_chain),
]

ragas_metrics = [
  LLMContextRecall(), 
  ContextEntityRecall(), 
]

# Generate test set data
generator = TestsetGenerator(
  llm=eval_gen_llm, 
  embedding_model=eval_gen_embeddings
)

In [ ]:
from ragas import RunConfig, EvaluationDataset, evaluate as ragas_evaluate

def evaluate_with_ragas(chain_name, dataset, chain, path):
  try:
    dataset_df = pd.read_json(path / "dataset.json")
    dataset = EvaluationDataset.from_pandas(dataset_df)
  except:
    for test_row in dataset:
      print(test_row)
      user_input = test_row.user_input
      print(f"Generating response for: {user_input}")
      response = chain.invoke({"question": user_input})
      test_row.response = response["response"].content
      test_row.retrieved_contexts = [context.page_content for context in response["context"]]
    print(f"Saving {chain_name} testset data to {path}/dataset.json")
    dataset.to_pandas().to_json(path / "dataset.json", orient="records", indent=4)

  result = ragas_evaluate(
      dataset=dataset,
      metrics=ragas_metrics,
      llm=eval_judge_llm,
      run_config=RunConfig(timeout=360)
  )
  print(f"{chain_name} results: {result}")

  print(f"Saving {chain_name} results to {path}/ragas_results.json")
  (path / "ragas_results.json").write_text(result.to_pandas().to_json(orient="records", indent=4))  
  

In [38]:
import getpass
from langsmith import Client as LangsmithClient
from langsmith.evaluation import evaluate as langsmith_evaluate

LANGSMITH_API_KEY = getpass.getpass("Enter your LangSmith API Key:")

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_PROJECT"] = "John Wick Movie Review RAG"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"

def evaluate_with_langsmith(chain_name, dataset, chain):
  langsmith_client = LangsmithClient(api_key=LANGSMITH_API_KEY)
  
  for test_row in dataset:
    DATASET_NAME = f"John Wick Movie Review RAG - {chain_name}"

    if langsmith_client.has_dataset(dataset_name=DATASET_NAME):
      langsmith_dataset = langsmith_client.read_dataset(dataset_name=DATASET_NAME)
    else:
      langsmith_dataset = langsmith_client.create_dataset(
          dataset_name=DATASET_NAME,
          description=f"John Wick movie review RAG with {chain_name}"
      )

    user_input = test_row.user_input
    print(f"Generating response for: {user_input}")
    response = chain.invoke({"question": user_input})
    langsmith_client.create_example(
      # example_id=chain_name,
      inputs={
        "question": user_input
      },
      outputs={
        "answer": response["response"].content,
        "context": response["context"],
      },
      metadata={
        "name": chain_name,
      },
      dataset_id=langsmith_dataset.id,
    )

  langsmith_evaluate(
    chain.invoke,
    data=DATASET_NAME,
    evaluators=[
        retrieval_relevance
    ],
    experiment_prefix="john-wick-rag",
    client=langsmith_client
  )

In [40]:
from ragas import EvaluationDataset

try:
  # load cached test set data
  dataset_df = pd.read_json("testset_data.json")
  dataset = EvaluationDataset.from_pandas(dataset_df)
except:
  # or generate new test set data
  print("Generating test set data...")
  dataset = generator.generate_with_langchain_docs(documents, testset_size=10)
  dataset.to_pandas().to_json("testset_data.json", orient="records", indent=4)
  
for chain_name, chain in chains:
  print(f"Evaluating chain: {chain_name}")
  path = Path("eval_results") / chain_name
  path.mkdir(parents=True, exist_ok=True)
  
  # copy dataset to avoid modifying the original
  chain_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())
  
  print("RAGAS Evaluation...")
  evaluate_with_ragas(chain_name, chain_dataset, chain, path)

  print("Langsmith Evaluation...")
  evaluate_with_langsmith(chain_name, chain_dataset, chain)


Evaluating chain: naive_retrieval
RAGAS Evaluation...


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

naive_retrieval results: {'context_recall': 0.8600, 'context_entity_recall': 0.6150}
Saving naive_retrieval results to eval_results/naive_retrieval/ragas_results.json
Langsmith Evaluation...
Generating response for: What makes John Wick a standout action film according to reviews?
Generating response for: Why might an action film aficionado be interested in watching John Wick?
Generating response for: Whut makes Chad Stahelski's direction in John Wick stand out in the action genre?
Generating response for: How do the Russian mobsters influence the plot development in the action film featuring John Wick?
Generating response for: What role does the Russian mob play in the movie John Wick?
Generating response for: How does the action choreography and pacing in John Wick 3 contribute to its acclaim, and how does this compare to the reception of the subsequent film in the franchise?
Generating response for: What are the differences in storytelling and action sequences between John Wick: Cha

0it [00:00, ?it/s]

Evaluating chain: bm25_retrieval
RAGAS Evaluation...


...

0it [00:00, ?it/s]

Evaluating chain: ensemble_retrieval
RAGAS Evaluation...


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

ensemble_retrieval results: {'context_recall': 0.8600, 'context_entity_recall': 0.5429}
Saving ensemble_retrieval results to eval_results/ensemble_retrieval/ragas_results.json
Langsmith Evaluation...
Generating response for: What makes John Wick a standout action film according to reviews?
Generating response for: Why might an action film aficionado be interested in watching John Wick?
Generating response for: Whut makes Chad Stahelski's direction in John Wick stand out in the action genre?
Generating response for: How do the Russian mobsters influence the plot development in the action film featuring John Wick?
Generating response for: What role does the Russian mob play in the movie John Wick?
Generating response for: How does the action choreography and pacing in John Wick 3 contribute to its acclaim, and how does this compare to the reception of the subsequent film in the franchise?
Generating response for: What are the differences in storytelling and action sequences between John 

0it [00:00, ?it/s]

## Retrieval Methods Comparison

|-----------------------------|---------------------|-------------|-------------|-------------|----------------|-----------------------|
| Method                      | Retrieval Relevance | P50 Latency | P99 Latency | Total Cost  | Context Recall | Context Entity Recall |
|-----------------------------|---------------------|-------------|-------------|-------------|----------------|-----------------------|
| Naive RAG                   | 1.0                 | ~1.7s       | ~3s         | ~$0.04      | 0.8100         | 0.5500                |
| Ensemble Retrieval          | 1.0                 | ~4.1s       | ~14.75s     | ~$0.06      | 0.8600         | 0.5300                |
| Multi-Query Retrieval       | 1.0                 | ~3.2s       | ~8.5s       | ~$0.05      | 0.8350         | 0.5333                |
| Parent Document Retrieval   | 0.95                | ~1.4s       | ~3s         | ~$0.01      | 0.5583         | 0.4767                |
| BM25 Retrieval              | 0.8                 | ~0.9s       | ~1.9s       | ~$0.01      | 0.4733         | 0.5350                |

Overall, Naive RAG is the best method for this particular dataset. It struck a nice balance between retrieval performance, cost, and latency. 

The Ensemble Retrieval method had the highest retrieval performance, but the latency and cost were higher than Naive RAG. This would be the best choice of method if retrieval performance was the most important factor. Retrieval performance was likely the highest because it leverages all of the advanced context retrieval methods.

Multi-Query Retrieval had the second highest retrieval performance, but the latency and cost were higher than Naive RAG. This would be a good choice of method if retrieval performance was important but so was cost and latency. Retrieval performance was likely the second highest because it leverages multiple queries to retrieve documents and likely handled the more subjective questions better by querying for a wider variety of matches.

Parent Document Retrieval surprisingly performed poorly in in Context Recall and Context Entity Recall. 

BM25 Retrieval was the worst method for this dataset. It had the lowest retrieval relevance scores, likely because it is based on a bag-of-words model which is not well suited to these types of questions.
